# ***YOLOv12 + MobileNetV3-small + FPN***

In [1]:
!pip install -q ultralytics roboflow

import os
from pathlib import Path
import yaml
from roboflow import Roboflow
from ultralytics import YOLO

# Konfigurasi Roboflow
API_KEY = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE = "dentalogic8"
PROJECT_ID = "dental-caries-7kttb"
VERSION_NUM = 10

# Download Dataset
rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"/content/dental-caries-{VERSION_NUM}")
DATA_YAML = DATASET_DIR / "data.yaml"

# Verifikasi download
if DATA_YAML.exists():
    print(f"✅ Dataset berhasil diunduh: {DATA_YAML}")
    print(f"Struktur dataset: {os.listdir(DATASET_DIR)}")
else:
    raise FileNotFoundError("Dataset gagal diunduh. Periksa koneksi dan kredensial Roboflow.")

# Custom YAML untuk YOLOv12 dengan MobileNetV3-Small backbone via TorchVision dan FPN neck
custom_yaml_content = """
nc: 7  # number of classes
scales:  # model compound scaling constants
  n: [0.50, 0.25, 1024]
  s: [0.50, 0.50, 1024]
  m: [0.50, 1.00, 512]
  l: [1.00, 1.00, 512]
  x: [1.00, 1.50, 512]

# MobileNetV3-Small backbone via TorchVision (adapted to match original YOLOv12 outputs)
backbone:
  # [from, repeats, module, args]
  - [-1, 1, TorchVision, [0, mobilenet_v3_small, DEFAULT, True, 2, True]]  # 0: features
  - [0, 1, Index, [24, 3]]  # 1: P3 raw (24ch, /8)
  - [1, 1, Conv, [256, 1, 1]]  # 2: P3 projected (256ch, /8)
  - [0, 1, Index, [48, 8]]  # 3: P4 raw (48ch, /16)
  - [3, 1, Conv, [512, 1, 1]]  # 4: P4 projected (512ch, /16)
  - [0, 1, Index, [96, 12]]  # 5: P5 raw (96ch, /32)
  - [5, 1, Conv, [1024, 1, 1]]  # 6: P5 projected (1024ch, /32)

# FPN neck (adapted from standard FPN with lateral convs, upsamples, concats, and C2f blocks)
head:
  # Lateral convs for P5, P4, P3 (project to common channels, e.g., 256 for 'n' scale)
  - [6, 1, Conv, [256, 1, 1]]  # 7: P5 lateral (1024->256)
  - [4, 1, Conv, [256, 1, 1]]  # 8: P4 lateral (512->256)
  - [2, 1, Conv, [256, 1, 1]]  # 9: P3 lateral (256->256)
  # Top-down pathway
  - [7, 1, nn.Upsample, [None, 2, 'nearest']]  # 10: up P5 lateral
  - [[-1, 8], 1, Concat, [1]]  # 11: cat up P5 + P4 lateral (256+256=512ch)
  - [-1, 2, C2f, [256, False]]  # 12: C2f P4_fpn (512->256)
  - [12, 1, nn.Upsample, [None, 2, 'nearest']]  # 13: up P4_fpn
  - [[-1, 9], 1, Concat, [1]]  # 14: cat up P4_fpn + P3 lateral (256+256=512ch)
  - [-1, 2, C2f, [256, False]]  # 15: C2f P3_fpn (512->256)
  # Bottom-up pathway (optional, but for PAN-like; here simple FPN)
  - [15, 1, Conv, [256, 3, 2]]  # 16: down P3_fpn to P4
  - [[-1, 12], 1, Concat, [1]]  # 17: cat down P3_fpn + P4_fpn (256+256=512ch)
  - [-1, 2, C2f, [512, False]]  # 18: C2f P4_pan (512->512)
  - [18, 1, Conv, [512, 3, 2]]  # 19: down P4_pan to P5
  - [[-1, 7], 1, Concat, [1]]  # 20: cat down P4_pan + P5 lateral (512+256=768ch)
  - [-1, 2, C2f, [512, False]]  # 21: C2f P5_pan (768->512)
  - [[15, 18, 21], 1, Detect, [nc]]  # Detect(P3=15, P4=18, P5=21)
"""

# Simpan YAML ke file
custom_yaml_path = Path("/content/yolo12n_mobilenetv3-small_fpn.yaml")
with open(custom_yaml_path, "w") as f:
    f.write(custom_yaml_content)
print(f"✅ Custom YAML disimpan: {custom_yaml_path}")

# Inisialisasi model dengan custom YAML
model = YOLO(str(custom_yaml_path))

def train_model(model, data_yaml, project_name="train_icdas_mnetv3-small_fpn", epochs=50):
    results = model.train(
        data=data_yaml,
        imgsz=640,
        epochs=epochs,
        batch=16,
        device=0,
        deterministic=True,
        project="runs/pure",
        name=project_name,
        pretrained=False  # Backbone pretrained via DEFAULT, head from scratch
    )
    return results

train_results = train_model(model, DATA_YAML)
print("✅ Training selesai! Model terbaik: runs/pure/train_icdas_mnetv3-small_fpn/weights/best.pt")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 126.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to dental-caries-9 in yolov12:: 100%|██████████| 13112/13112 [00:02<00:00, 6307.83it/s]


✅ Dataset berhasil diunduh: /content/dental-caries-9/data.yaml
Struktur dataset: ['train', 'valid', 'README.dataset.txt', 'data.yaml', 'README.roboflow.txt', 'test']
✅ Custom YAML disimpan: /content/yolo12n_mobilenetv3-small_fpn.yaml
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 132MB/s]


Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dental-caries-9/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/yolo12n_mobilenetv3-small_fpn.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train_icdas_mnetv3-small_fpn, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

KeyboardInterrupt: 